# Python 모듈 or 노트북 파일
# 1. 회사 보고서 작성 에이전트 (RAG 회사 자료)

In [1]:
import os
from dotenv import load_dotenv
from pprint import pprint
from pydantic import BaseModel, Field 
from pathlib import Path
from typing import List, Dict, Any
import json
import pandas as pd
import sqlite3

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder 
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser
from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.utilities import SQLDatabase
# from langchain_community.document_loaders import UnstructuredMarkdownLoader <-- 이 줄을 주석 처리하거나 삭제
from langchain_community.document_loaders import TextLoader # <-- 이 줄을 추가
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub
from langchain.memory import ConversationBufferMemory





# 환경 변수를 로드합니다. .env 파일에 OPENAI_API_KEY가 설정되어 있어야 합니다.
load_dotenv()

True

In [2]:
llm = ChatOpenAI(model='gpt-4.1-nano',temperature=0.5)

# 메모리 객체 초기화
# 각 에이전트는 독립적인 대화 기록을 가져야 하므로, 필요시 별도의 메모리 객체를 생성합니다.
# 여기서는 각 에이전트를 생성할 때 메모리를 주입하는 패턴을 사용합니다.
# 대화 기록을 저장할 변수명은 'chat_history'로 지정합니다.

print("공통 LLM 모델이 초기화되었습니다.")

공통 LLM 모델이 초기화되었습니다.


In [3]:
# 요구사항: 프롬프트 -> JSON 변환, OutputParser 활용

# 1. Pydantic 모델 정의 (사용자님이 주신 모델 활용)
class CsvCreationInput(BaseModel):
    """CSV 생성을 위한 데이터 구조"""
    file_name: str = Field(description="사용자 요청에 기반한 논리적인 CSV 파일 이름. 예: 'customers.csv'")
    data: List[Dict[str, Any]] = Field(description="사용자 요청에 따라 생성된 샘플 데이터. 딕셔너리의 리스트 형태여야 합니다.")

# 2. Output Parser 설정
# Pydantic 모델을 기반으로 JSON 출력을 검증하고 파싱하는 파서를 만듭니다.
json_parser = JsonOutputParser(pydantic_object=CsvCreationInput)

# 3. 프롬프트 템플릿 설정
# 이 프롬프트의 유일한 목표는 Pydantic 모델 형식에 맞는 JSON을 생성하는 것입니다.
prompt_to_json = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an expert data generation assistant. Your task is to generate structured data based on the user's request. "
     "You must generate output in a JSON format that strictly follows the provided schema. "
     "Infer column names and create realistic sample data.\n"
     "{format_instructions}"),
    ("user", "{user_prompt}")
])

# 4. 체인(Chain) 생성
# LangChain Expression Language (LCEL)을 사용하여 구성요소를 연결합니다.
json_generation_chain = prompt_to_json | llm | json_parser

print("1단계: 프롬프트 -> JSON 변환 체인이 생성되었습니다.")
print("\n--- Pydantic 스키마에 기반한 JSON 포맷 지시사항 ---")
pprint(json_parser.get_format_instructions())

1단계: 프롬프트 -> JSON 변환 체인이 생성되었습니다.

--- Pydantic 스키마에 기반한 JSON 포맷 지시사항 ---
('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"description": "CSV 생성을 위한 데이터 구조", "properties": {"file_name": '
 '{"description": "사용자 요청에 기반한 논리적인 CSV 파일 이름. 예: \'customers.csv\'", "title": '
 '"File Name", "type": "string"}, "data": {"description": "사용자 요청에 따라 생성된 샘플 '
 '데이터. 딕셔너리의 리스트 형태여야 합니다.", "items": {"additionalProperties": true, "type": '
 '"object"}, "title": "Data", "type": "array"}}, "required": ["file_name", '
 '"data"]}\n'
 '```')


In [4]:
# 요구사항: 프롬프트 -> JSON 변환, OutputParser 활용

# 1. Pydantic 모델 정의
class CsvCreationInput(BaseModel):
    """CSV 생성을 위한 데이터 구조"""
    file_name: str = Field(description="사용자 요청에 기반한 논리적인 CSV 파일 이름. 예: 'customers.csv'")
    data: List[Dict[str, Any]] = Field(description="사용자 요청에 따라 생성된 샘플 데이터. 딕셔너리의 리스트 형태여야 합니다.")

# 2. Output Parser 설정
json_parser = JsonOutputParser(pydantic_object=CsvCreationInput)

# 3. 프롬프트 템플릿 설정
prompt_to_json = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an expert data generation assistant. Your task is to generate structured data based on the user's request. "
     "You must generate output in a JSON format that strictly follows the provided schema. "
     "Infer column names and create realistic sample data.\n"
     "{format_instructions}"), # Output Parser가 제공하는 형식 지시사항
    ("user", "{user_prompt}")
])

# 4. 체인(Chain) 생성 (<<<<<<<<<<< 여기가 핵심 수정 사항입니다! >>>>>>>>>>>)
# .partial()을 사용하여 format_instructions 변수를 미리 채워넣습니다.
# 이렇게 하면 체인을 호출할 때는 user_prompt만 전달하면 됩니다.
json_generation_chain = prompt_to_json.partial(
    format_instructions=json_parser.get_format_instructions()
) | llm | json_parser

print("✅ 1단계: 프롬프트 -> JSON 변환 체인이 생성되었습니다.")
print("\n--- Pydantic 스키마에 기반한 JSON 포맷 지시사항 (프롬프트에 자동으로 주입됨) ---")
pprint(json_parser.get_format_instructions())

✅ 1단계: 프롬프트 -> JSON 변환 체인이 생성되었습니다.

--- Pydantic 스키마에 기반한 JSON 포맷 지시사항 (프롬프트에 자동으로 주입됨) ---
('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"description": "CSV 생성을 위한 데이터 구조", "properties": {"file_name": '
 '{"description": "사용자 요청에 기반한 논리적인 CSV 파일 이름. 예: \'customers.csv\'", "title": '
 '"File Name", "type": "string"}, "data": {"description": "사용자 요청에 따라 생성된 샘플 '
 '데이터. 딕셔너리의 리스트 형태여야 합니다.", "items": {"additionalProperties": true, "type": '
 '"object"}, "title": "Data", "type": "array"}}, "required": ["file_name", '
 '"data"]}\n'
 '

In [5]:
# 1단계 체인을 사용하여 사용자의 자연어 요청을 JSON으로 변환합니다.
user_request = "신규 고객 명단 3개를 샘플로 만들어줘. '이름', '이메일', '가입일' 컬럼이 필요해. 파일명은 customers.csv로 해줘."

# 체인 실행
generated_json = json_generation_chain.invoke({
    "user_prompt": user_request,
    "format_instructions": json_parser.get_format_instructions()
})

print("--- 1단계 실행 결과 (생성된 JSON) ---")
pprint(generated_json)

# 생성된 JSON을 다음 단계를 위해 상태 저장소에 저장합니다.
app_state = {
    "generated_json": generated_json,
    "dataframe": None,
    "chat_history": []
}
print("\n상태 저장소(app_state)에 JSON 결과가 저장되었습니다.")

--- 1단계 실행 결과 (생성된 JSON) ---
{'data': [{'가입일': '2024-02-15', '이름': '김민수', '이메일': 'minsu.kim@example.com'},
          {'가입일': '2024-02-16', '이름': '이수진', '이메일': 'sujin.lee@example.com'},
          {'가입일': '2024-02-17', '이름': '박준호', '이메일': 'junho.park@example.com'}],
 'file_name': 'customers.csv'}

상태 저장소(app_state)에 JSON 결과가 저장되었습니다.


In [6]:
# 요구사항: JSON을 받아 처리하는 Tool, 추가 프롬프트 기반으로 Tool 실행 판단

# --- 상태 관리 ---
# 에이전트가 Tool을 통해 이 app_state 딕셔너리를 읽고 수정하도록 설계합니다.

# --- Tool 정의 ---
@tool
def save_json_as_csv() -> str:
    """
    현재 상태(app_state)에 저장된 JSON 데이터를 사용하여 CSV 파일을 저장합니다.
    이 Tool은 별도의 인자가 필요 없으며, 상태에 있는 'generated_json'을 사용합니다.
    """
    json_data = app_state.get("generated_json")
    if not json_data:
        return "오류: 먼저 JSON 데이터를 생성해야 합니다."
    
    try:
        file_name = json_data.get('file_name', 'output.csv')
        data = json_data.get('data', [])
        
        if not data:
            return "오류: 데이터가 비어있어 CSV 파일을 생성할 수 없습니다."

        df = pd.DataFrame(data)
        app_state["dataframe"] = df  # 데이터프레임을 상태에 저장
        
        output_dir = Path("data")
        output_dir.mkdir(exist_ok=True)
        file_path = output_dir / file_name
        
        df.to_csv(file_path, index=False, encoding='utf-8-sig')
        return f"성공: 데이터가 '{file_path}'에 저장되었고, DataFrame이 상태에 저장되었습니다."
    except Exception as e:
        return f"오류 발생: {e}"

@tool
def add_column(column_name: str, default_value: Any) -> str:
    """
    현재 상태(app_state)에 저장된 DataFrame에 새로운 컬럼을 추가합니다.
    """
    df = app_state.get("dataframe")
    if df is None:
        return "오류: 먼저 DataFrame을 생성해야 합니다. (save_json_as_csv를 먼저 실행하세요)"
    
    df[column_name] = default_value
    app_state["dataframe"] = df # 수정된 DataFrame을 다시 상태에 저장
    return f"성공: '{column_name}' 컬럼이 추가되었습니다. 현재 DataFrame을 확인하세요."

# --- 에이전트 설정 ---
action_tools = [save_json_as_csv, add_column]

# 이 에이전트는 '행동'에만 집중합니다.
action_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that acts on existing data. Use the available tools to fulfill the user's request."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

action_agent = create_openai_tools_agent(llm, action_tools, action_prompt)

action_agent_executor = AgentExecutor(
    agent=action_agent,
    tools=action_tools,
    verbose=True
)

print("2/3단계: 행동(Action) 에이전트 및 Tools가 생성되었습니다.")

2/3단계: 행동(Action) 에이전트 및 Tools가 생성되었습니다.


In [7]:
# 요구사항: JSON을 받아 처리하는 Tool, 추가 프롬프트 기반으로 Tool 실행 판단

# --- 상태 관리 ---
app_state = {
    "generated_json": None,
    "dataframe": None
}
print("✅ 상태 관리 객체 'app_state'가 초기화되었습니다.")

# --- Tool 정의 ---
@tool
def save_json_as_csv() -> str:
    """
    현재 상태(app_state)에 저장된 JSON 데이터를 사용하여 CSV 파일을 저장하고,
    생성된 DataFrame을 상태에 저장합니다.
    """
    json_data = app_state.get("generated_json")
    if not json_data:
        return "오류: 먼저 1단계에서 JSON 데이터를 생성해야 합니다."
    
    try:
        file_name = json_data.get('file_name', 'output.csv')
        data = json_data.get('data', [])
        if not data:
            return "오류: 데이터가 비어있어 CSV 파일을 생성할 수 없습니다."

        df = pd.DataFrame(data)
        app_state["dataframe"] = df
        
        output_dir = Path("data")
        output_dir.mkdir(exist_ok=True)
        file_path = output_dir / file_name
        
        df.to_csv(file_path, index=False, encoding='utf-8-sig')
        return f"성공: 데이터가 '{file_path}'에 저장되었고, DataFrame이 내부 상태에 저장되었습니다."
    except Exception as e:
        return f"오류 발생: {e}"

@tool
def add_column(column_name: str, default_value: Any) -> str:
    """
    현재 상태(app_state)에 저장된 DataFrame에 새로운 컬럼을 추가합니다.
    """
    df = app_state.get("dataframe")
    if df is None:
        return "오류: 먼저 DataFrame을 생성해야 합니다."
    
    df[column_name] = default_value
    app_state["dataframe"] = df
    return f"성공: '{column_name}' 컬럼이 추가되었습니다."

# <<<<<<<<<<< 여기가 새로 추가된 부분입니다! >>>>>>>>>>>
@tool
def check_dataframe_status() -> str:
    """
    현재 app_state에 저장된 DataFrame의 상태를 확인하고,
    데이터가 존재하면 처음 5개 행(head)을 문자열로 반환합니다.
    데이터의 현재 상태를 확인하거나 보여달라는 요청에 사용하세요.
    """
    df = app_state.get("dataframe")
    if df is None:
        return "현재 상태에 DataFrame이 존재하지 않습니다. 먼저 데이터를 생성하고 저장해주세요."
    else:
        return f"현재 DataFrame이 존재합니다. 처음 5개 행은 다음과 같습니다:\n{df.head().to_string()}"
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

print("✅ 2/3단계를 위한 Tools가 생성되었습니다.")

✅ 상태 관리 객체 'app_state'가 초기화되었습니다.
✅ 2/3단계를 위한 Tools가 생성되었습니다.


In [8]:
# --- 에이전트 설정 ---
# <<<<<<<<<<< 여기가 수정된 부분입니다! (새로운 tool 추가) >>>>>>>>>>>
action_tools = [save_json_as_csv, add_column, check_dataframe_status]
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

action_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that acts on existing data. Use the available tools to fulfill the user's request. Do not generate new data yourself."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

action_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

action_agent = create_openai_tools_agent(llm, action_tools, action_prompt)

action_agent_executor = AgentExecutor(
    agent=action_agent,
    tools=action_tools,
    verbose=True,
    memory=action_memory
)

print("✅ 2/3단계: 행동(Action) 에이전트가 **업그레이드** 되었습니다.")

✅ 2/3단계: 행동(Action) 에이전트가 **업그레이드** 되었습니다.


/var/folders/1p/yb8d7hmx6wd564h57_td4k140000gn/T/ipykernel_42523/2433834999.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  action_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [9]:
# --- 1단계 실행: 프롬프트 -> JSON ---
print("--- [1단계] 프롬프트로 JSON 데이터 생성 시작 ---")
user_request_for_json = "신규 고객 명단 3개를 샘플로 만들어줘. '이름', '이메일', '가입일' 컬럼이 필요해."
generated_json = json_generation_chain.invoke({"user_prompt": user_request_for_json})

# 생성된 JSON을 상태에 저장
app_state["generated_json"] = generated_json

print("\n--- [1단계] JSON 생성 완료 ---")
pprint(app_state["generated_json"])


# --- 2단계 실행: JSON -> CSV (에이전트 판단) ---
print("\n\n--- [2단계] 생성된 JSON을 CSV로 저장하도록 에이전트에게 요청 ---")
user_request_for_action = "방금 만든 JSON을 CSV 파일로 저장해줘."
response_csv = action_agent_executor.invoke({"input": user_request_for_action})
print("\n--- [2단계] 에이전트 응답 ---")
pprint(response_csv['output'])

print("\n--- [2단계] 작업 후 DataFrame 상태 확인 ---")
print(app_state["dataframe"])


# --- 3단계 실행: DataFrame 수정 (에이전트 판단) ---
print("\n\n--- [3단계] DataFrame에 컬럼을 추가하도록 에이전트에게 요청 ---")
user_request_for_modification = "생성된 데이터에 '고객등급' 컬럼을 추가하고 기본값으로 'Bronze'를 넣어줘."
response_add_col = action_agent_executor.invoke({"input": user_request_for_modification})
print("\n--- [3단계] 에이전트 응답 ---")
pprint(response_add_col['output'])

print("\n--- [3단계] 최종 DataFrame 상태 확인 ---")
print(app_state["dataframe"])

--- [1단계] 프롬프트로 JSON 데이터 생성 시작 ---

--- [1단계] JSON 생성 완료 ---
{'data': [{'가입일': '2024-04-01', '이름': '김민수', '이메일': 'minsu.kim@example.com'},
          {'가입일': '2024-04-02', '이름': '이수진', '이메일': 'sujin.lee@example.com'},
          {'가입일': '2024-04-03', '이름': '박지훈', '이메일': 'jihoon.park@example.com'}],
 'file_name': 'new_customers.csv'}


--- [2단계] 생성된 JSON을 CSV로 저장하도록 에이전트에게 요청 ---


> Entering new AgentExecutor chain...

Invoking: `save_json_as_csv` with `{}`


성공: 데이터가 'data/new_customers.csv'에 저장되었고, DataFrame이 내부 상태에 저장되었습니다.JSON 데이터를 CSV 파일로 성공적으로 저장했습니다. 파일 이름은 'data/new_customers.csv'입니다. 추가로 필요한 것이 있나요?

> Finished chain.

--- [2단계] 에이전트 응답 ---
("JSON 데이터를 CSV 파일로 성공적으로 저장했습니다. 파일 이름은 'data/new_customers.csv'입니다. 추가로 필요한 "
 '것이 있나요?')

--- [2단계] 작업 후 DataFrame 상태 확인 ---
    이름                      이메일         가입일
0  김민수    minsu.kim@example.com  2024-04-01
1  이수진    sujin.lee@example.com  2024-04-02
2  박지훈  jihoon.park@example.com  2024-04-03


--- [3단계] DataFrame에 컬럼을 추가하도록 에이전트에게 

In [10]:
# --- 라우터(Router) 체인 설정 ---
# 사용자의 의도를 'generation'(생성) 또는 'action'(행동)으로 분류합니다.

router_prompt_template = """당신은 사용자의 요청을 두 가지 카테고리 중 하나로 분류하는 라우터입니다.
'generation' 또는 'action' 중 하나로만 대답해야 합니다.

'generation': 사용자가 처음부터 새로운 데이터를 만들어달라고 요청하는 경우.
(예: "고객 명단 만들어줘", "제품 5개 리스트 생성해줘")

'action': 사용자가 이미 생성된 데이터를 가지고 무언가를 해달라고 요청하는 경우.
(예: "CSV로 저장해", "방금 만든거에 컬럼 추가해줘", "데이터 수정해줘")

사용자 요청: {user_prompt}
분류:"""

router_prompt = ChatPromptTemplate.from_template(router_prompt_template)
router_chain = router_prompt | llm | StrOutputParser()

print("✅ 대화형 테스트를 위한 라우터가 준비되었습니다.")
print("--- 채팅을 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요. ---")

# --- 대화 루프 시작 ---
while True:
    try:
        # 사용자 입력 받기
        user_input = input("You: ")

        # 종료 조건 확인
        if user_input.lower() in ["exit", "quit", "종료"]:
            print("Bot: 대화를 종료합니다.")
            break

        # 1. 라우터를 통해 사용자의 의도 파악
        intent = router_chain.invoke({"user_prompt": user_input})
        print(f"[시스템] 의도 분석 결과: '{intent}'")

        # 2. 의도에 따라 적절한 체인 또는 에이전트 실행
        if "generation" in intent.lower():
            print("\n[시스템] 1단계: JSON 생성 체인을 실행합니다...")
            # 1단계 실행: 프롬프트 -> JSON
            generated_json = json_generation_chain.invoke({"user_prompt": user_input})
            app_state["generated_json"] = generated_json
            app_state["dataframe"] = None # 새로운 데이터가 생성되었으므로 DataFrame은 초기화
            
            print("\n--- [1단계] JSON 생성 완료 ---")
            pprint(app_state["generated_json"])
            print("\nBot: JSON 데이터가 성공적으로 생성되었습니다. 이제 '저장해줘' 또는 '컬럼 추가해줘' 같은 요청을 할 수 있습니다.")

        elif "action" in intent.lower():
            # 행동을 하기 전에, 생성된 데이터가 있는지 확인
            if app_state.get("generated_json") is None:
                print("\nBot: 앗, 아직 조작할 데이터가 없어요. 먼저 데이터를 생성해 주세요. (예: '고객 3명의 명단을 만들어줘')")
                continue

            print("\n[시스템] 2/3단계: 행동 에이전트를 실행합니다...")
            # 2/3단계 실행: JSON -> CSV 또는 DataFrame 수정
            response = action_agent_executor.invoke({"input": user_input})
            
            print("\n--- 에이전트 응답 ---")
            pprint(response['output'])

            # DataFrame이 변경되었을 수 있으니 현재 상태를 보여줌
            if app_state.get("dataframe") is not None:
                print("\n--- 현재 DataFrame 상태 ---")
                print(app_state["dataframe"])
        
        else:
            print("Bot: 죄송합니다. 요청의 의도를 파악하지 못했습니다. 데이터를 새로 생성하시겠어요, 아니면 기존 데이터를 조작하시겠어요?")

    except Exception as e:
        print(f"\n[오류 발생] 오류가 발생했습니다: {e}")
        print("Bot: 죄송합니다. 처리 중 오류가 발생했습니다. 다른 요청을 시도해 주세요.")

✅ 대화형 테스트를 위한 라우터가 준비되었습니다.
--- 채팅을 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요. ---
Bot: 대화를 종료합니다.


In [11]:
app_state['dataframe']

,이름,이메일,가입일,고객등급
0,김민수,minsu.kim@example.com,2024-04-01,Bronze
1,이수진,sujin.lee@example.com,2024-04-02,Bronze
2,박지훈,jihoon.park@example.com,2024-04-03,Bronze


In [12]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from scipy.stats import ttest_ind

In [13]:
%pip install langchain-experimental scipy tabulate


Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from scipy.stats import ttest_ind

def create_analysis_agent_executor(df: pd.DataFrame):
    if df is None:
        return None
    
    analysis_agent_executor = create_pandas_dataframe_agent(
        llm,
        df,
        verbose=True,
        # <<<<<<<<<<< 여기가 수정된 부분입니다! >>>>>>>>>>>
        allow_dangerous_code=True, # 위험한 코드 실행을 명시적으로 허용
        # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        agent_executor_kwargs={"handle_parsing_errors": True}
    )
    return analysis_agent_executor

print("✅ 4단계: 데이터 분석(Analysis) 에이전트 생성 함수가 준비되었습니다. (안전 옵션 활성화)")

✅ 4단계: 데이터 분석(Analysis) 에이전트 생성 함수가 준비되었습니다. (안전 옵션 활성화)


In [15]:
# 셀 8

# --- 최종 대화형 애플리케이션 실행 루프 (안내 기능 강화) ---

print("✅ 최종 버전의 대화형 라우터가 준비되었습니다.")
print("--- 채팅을 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요. ---")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "종료","정지","그만"]:
            print("Bot: 대화를 종료합니다.")
            break

        intent = router_chain.invoke({"user_prompt": user_input})
        print(f"\n[시스템] 의도 분석 결과: '{intent}'")

        if "generation" in intent.lower():
            print("\n[시스템] 1단계: JSON 생성 체인을 실행합니다...")
            generated_json = json_generation_chain.invoke({"user_prompt": user_input})
            app_state["generated_json"] = generated_json
            app_state["dataframe"] = None
            print("\n--- [1단계] JSON 생성 완료 ---")
            pprint(app_state["generated_json"])
            print("\nBot: JSON 데이터가 생성되었습니다. '저장해줘' 또는 구체적인 분석 요청을 할 수 있습니다.")

        elif "action" in intent.lower():
            if app_state.get("generated_json") is None:
                print("\nBot: 앗, 아직 조작할 데이터가 없어요. 먼저 데이터를 생성해 주세요.")
                continue
            print("\n[시스템] 2/3단계: 행동 에이전트를 실행합니다...")
            response = action_agent_executor.invoke({"input": user_input})
            print("\n--- 에이전트 응답 ---")
            pprint(response['output'])
            if app_state.get("dataframe") is not None:
                print("\n--- 현재 DataFrame 상태 ---")
                print(app_state["dataframe"])

        elif "analysis" in intent.lower():
            # <<<<<<<<<<< 여기가 수정된 부분입니다! (친절한 안내 추가) >>>>>>>>>>>
            if app_state.get("dataframe") is None:
                if app_state.get("generated_json") is not None:
                    print("\nBot: 앗, 데이터를 분석하려면 먼저 '저장해줘'라고 말씀하셔서 DataFrame으로 변환해야 해요.")
                else:
                    print("\nBot: 앗, 아직 분석할 데이터가 없어요. 먼저 데이터를 생성해주세요.")
                continue
            # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
            
            print("\n[시스템] 4단계: 데이터 분석 에이전트를 실행합니다...")
            analysis_agent = create_analysis_agent_executor(app_state["dataframe"])
            response = analysis_agent.invoke({"input": user_input})
            
            # <<<<<<<<<<< 여기가 수정된 부분입니다! (무한 루프 에러 처리) >>>>>>>>>>>
            if "Agent stopped due to iteration limit" in response.get('output', ''):
                print("\n--- 분석 에이전트 응답 ---")
                print("\nBot: 죄송합니다. 요청이 너무 광범위하여 분석 작업을 완료하지 못했습니다. '평균 병상 수는 얼마야?' 또는 '가장 오래된 병원은 어디야?'와 같이 더 구체적으로 질문해주시겠어요?")
            else:
                print("\n--- 분석 에이전트 응답 ---")
                pprint(response['output'])
            # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        
        else:
            print("Bot: 죄송합니다. 요청의 의도를 파악하지 못했습니다.")

    except Exception as e:
        print(f"\n[오류 발생] 오류가 발생했습니다: {e}")
        print("Bot: 죄송합니다. 처리 중 오류가 발생했습니다.")

✅ 최종 버전의 대화형 라우터가 준비되었습니다.
--- 채팅을 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요. ---
Bot: 대화를 종료합니다.


In [23]:
# ==============================================================================
# --- 셀 1: 전체 라이브러리 임포트 ---
# ==============================================================================
# %pip install -qU langchain langchain-openai pandas python-dotenv langchain_experimental scipy tabulate faiss-cpu "unstructured[md]"
import os, json, pandas as pd
from dotenv import load_dotenv
from pprint import pprint
from pathlib import Path
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# ==============================================================================
# --- 셀 2: 초기 설정 및 상태 관리 ---
# ==============================================================================
load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0)
app_state = {"generated_json": None, "dataframe": None, "rag_retriever": None}
action_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
print("✅ 초기 설정 및 상태 관리가 완료되었습니다.")

# ==============================================================================
# --- 셀 3: 1단계 - JSON 생성 체인 ---
# ==============================================================================
class CsvCreationInput(BaseModel):
    file_name: str = Field(description="CSV 파일 이름.")
    data: List[Dict[str, Any]] = Field(description="샘플 데이터 (딕셔너리 리스트).")
json_parser = JsonOutputParser(pydantic_object=CsvCreationInput)
prompt_to_json = ChatPromptTemplate.from_messages([
    ("system", "You are an expert data generation assistant. Generate structured data in a JSON format that strictly follows the provided schema.\n{format_instructions}"),
    ("user", "{user_prompt}")
])
json_generation_chain = prompt_to_json.partial(format_instructions=json_parser.get_format_instructions()) | llm | json_parser
print("✅ 1단계: JSON 생성 체인이 준비되었습니다.")

# ==============================================================================
# --- 셀 4: 모든 행동(Action) Tools 정의 ---
# ==============================================================================
@tool
def save_json_as_csv() -> str:
    """현재 상태의 JSON 데이터를 CSV 파일로 저장하고, DataFrame을 생성합니다."""
    json_data = app_state.get("generated_json")
    if not json_data: return "오류: 먼저 JSON 데이터를 생성해야 합니다."
    try:
        df = pd.DataFrame(json_data.get('data', []))
        app_state["dataframe"] = df
        file_path = Path("data") / json_data.get('file_name', 'output.csv')
        file_path.parent.mkdir(exist_ok=True)
        df.to_csv(file_path, index=False, encoding='utf-8-sig')
        return f"성공: 데이터가 '{file_path}'에 저장되었고, DataFrame이 생성되었습니다."
    except Exception as e: return f"오류 발생: {e}"

@tool
def add_column(column_name: str, default_value: Any) -> str:
    """현재 DataFrame에 새로운 컬럼을 추가합니다."""
    df = app_state.get("dataframe")
    if df is None: return "오류: 먼저 DataFrame을 생성해야 합니다."
    df[column_name] = default_value
    app_state["dataframe"] = df
    return f"성공: '{column_name}' 컬럼이 추가되었습니다."

@tool
def check_dataframe_status() -> str:
    """현재 DataFrame의 상태를 확인합니다."""
    df = app_state.get("dataframe")
    if df is None: return "현재 상태에 DataFrame이 존재하지 않습니다."
    return f"현재 DataFrame이 존재합니다. 처음 5개 행은 다음과 같습니다:\n{df.head().to_string()}"

@tool
def load_markdown_and_create_retriever(file_path: str) -> str:
    """지정된 마크다운(MD) 파일을 읽고 질의응답을 위한 RAG Retriever를 생성합니다."""
    try:
        loader = TextLoader(file_path, encoding='utf-8')
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(documents)
        vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
        app_state["rag_retriever"] = vectorstore.as_retriever()
        return f"성공: '{file_path}' 파일을 읽고 질의응답 준비를 마쳤습니다."
    except Exception as e: return f"오류: 파일을 읽는 중 문제가 발생했습니다. 원인: {e}"

@tool
def extract_data_from_document(user_query: str) -> str:
    """이미 로드된 문서에서 사용자가 요청한 정보를 추출하여 구조화된 JSON 데이터를 생성하고, 그 결과를 app_state에 저장합니다."""
    retriever = app_state.get("rag_retriever")
    if retriever is None: return "오류: 먼저 문서를 읽어야 합니다. '...파일 읽어줘'를 먼저 요청하세요."
    
    extraction_parser = JsonOutputParser(pydantic_object=CsvCreationInput)
    extraction_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert at extracting structured information from text. Analyze the provided context and extract the data requested by the user into a JSON format that strictly follows the schema.\n{format_instructions}"),
        ("user", "Context:\n{context}\n\nUser Request: {user_prompt}")
    ])
    extraction_chain = extraction_prompt.partial(format_instructions=extraction_parser.get_format_instructions()) | llm | extraction_parser
    
    try:
        full_context = "\n".join([doc.page_content for doc in retriever.get_relevant_documents(user_query, k=5)])
        extracted_json = extraction_chain.invoke({"user_prompt": user_query, "context": full_context})
        app_state["generated_json"] = extracted_json
        return f"성공: 문서에서 데이터를 성공적으로 추출하여 내부 상태에 저장했습니다. 추출된 내용:\n{json.dumps(extracted_json, indent=2, ensure_ascii=False)}"
    except Exception as e: return f"오류: 데이터 추출 중 문제가 발생했습니다. 원인: {e}"

action_tools = [save_json_as_csv, add_column, check_dataframe_status, load_markdown_and_create_retriever, extract_data_from_document]
print("✅ 모든 행동(Action) Tools가 정의되었습니다.")

# ==============================================================================
# --- 셀 5: 행동(Action) 에이전트 생성 ---
# ==============================================================================
action_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a manager assistant. Your job is to manage files and data states by calling the appropriate tools in the correct order. You can also extract structured data from documents."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
action_agent = create_openai_tools_agent(llm, action_tools, action_prompt)
action_agent_executor = AgentExecutor(agent=action_agent, tools=action_tools, verbose=True, memory=action_memory)
print("✅ 행동(Action) 에이전트가 준비되었습니다.")

# ==============================================================================
# --- 셀 6: 분석 및 Q&A 생성 함수 ---
# ==============================================================================
def create_analysis_agent_executor(df: pd.DataFrame):
    if df is None: return None
    return create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True, agent_executor_kwargs={"handle_parsing_errors": True})

def create_document_qa_chain(retriever):
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful Q&A assistant. Answer the user's question based only on the provided context."),
        ("user", "Context:\n{context}\n\nQuestion: {input}")
    ])
    return {"context": retriever, "input": RunnablePassthrough()} | rag_prompt | llm | StrOutputParser()
print("✅ 분석 및 Q&A 생성 함수가 준비되었습니다.")

# ==============================================================================
# --- 셀 7: 최종 통합 대화형 애플리케이션 ---
# ==============================================================================
router_prompt_template = """당신은 사용자의 요청을 4가지 카테고리로 분류하는 라우터입니다.
'generation', 'action', 'analysis', 'document_qa' 중 하나로만 대답해야 합니다.

'generation': 새로운 데이터를 처음부터 만들어달라는 요청. (예: "병원 데이터 5개 만들어줘")
'action': 파일이나 데이터 상태를 관리/조작/추출하는 모든 요청. (예: "CSV로 저장해", "컬럼 추가해줘", "커피 로그 파일 읽어줘", "커피 로그에서 데이터 추출해줘")
'analysis': DataFrame의 내용에 대해 질문/분석을 요청. (예: "평균 병상 수는?", "가장 오래된 병원은?")
'document_qa': 로드된 문서의 내용에 대해 질문. (예: "커피 로그 요약해줘", "추출 노트가 뭐야?")

사용자 요청: {user_prompt}
분류:"""
router_chain = ChatPromptTemplate.from_template(router_prompt_template) | llm | StrOutputParser()
print("✅ 최종 통합 라우터가 준비되었습니다.\n--- 채팅을 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요. ---")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "종료"]: print("Bot: 대화를 종료합니다."); break

        intent = router_chain.invoke({"user_prompt": user_input})
        print(f"\n[시스템] 의도 분석 결과: '{intent}'")

        if "generation" in intent.lower():
            print("\n[시스템] 1단계: JSON 생성 체인을 실행합니다...")
            generated_json = json_generation_chain.invoke({"user_prompt": user_input})
            app_state["generated_json"], app_state["dataframe"], app_state["rag_retriever"] = generated_json, None, None
            print("\n--- [1단계] JSON 생성 완료 ---"); pprint(app_state["generated_json"])
            print("\nBot: JSON 데이터가 생성되었습니다.")

        elif "action" in intent.lower():
            print("\n[시스템] 행동 에이전트를 실행합니다...")
            response = action_agent_executor.invoke({"input": user_input})
            print("\n--- 에이전트 응답 ---"); pprint(response['output'])

        elif "analysis" in intent.lower():
            if app_state.get("dataframe") is None: print("\nBot: 앗, 분석할 데이터가 없어요. 먼저 데이터를 생성/추출하고 '저장해줘'를 실행해주세요."); continue
            print("\n[시스템] 분석 에이전트를 실행합니다...")
            analysis_agent = create_analysis_agent_executor(app_state["dataframe"])
            response = analysis_agent.invoke({"input": user_input})
            if "Agent stopped" in response.get('output', ''): print("\n--- 분석 에이전트 응답 ---\nBot: 죄송합니다. 요청이 너무 광범위합니다. 더 구체적으로 질문해주세요.")
            else: print("\n--- 분석 에이전트 응답 ---"); pprint(response['output'])

        elif "document_qa" in intent.lower():
            retriever = app_state.get("rag_retriever")
            if retriever is None: print("\nBot: 앗, 질의응답할 문서가 없어요. 먼저 문서를 읽어주세요."); continue
            print("\n[시스템] 문서 Q&A 체인을 실행합니다...")
            doc_qa_chain = create_document_qa_chain(retriever)
            response = doc_qa_chain.invoke(user_input)
            print("\n--- 문서 Q&A 응답 ---\nBot:", response)
        
        else: print("Bot: 죄송합니다. 요청의 의도를 파악하지 못했습니다.")
    except Exception as e: print(f"\n[오류 발생]: {e}\nBot: 죄송합니다. 처리 중 오류가 발생했습니다.")

✅ 초기 설정 및 상태 관리가 완료되었습니다.
✅ 1단계: JSON 생성 체인이 준비되었습니다.
✅ 모든 행동(Action) Tools가 정의되었습니다.
✅ 행동(Action) 에이전트가 준비되었습니다.
✅ 분석 및 Q&A 생성 함수가 준비되었습니다.
✅ 최종 통합 라우터가 준비되었습니다.
--- 채팅을 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요. ---

[시스템] 의도 분석 결과: ''generation''

[시스템] 1단계: JSON 생성 체인을 실행합니다...

[오류 발생]: Invalid json output: 물론입니다. 주어진 JSON 스키마는 두 가지 주요 속성을 요구합니다:

1. `file_name`: 이 속성은 CSV 파일의 이름을 나타내는 문자열입니다. 이 속성은 필수입니다.

2. `data`: 이 속성은 샘플 데이터를 나타내며, 딕셔너리(객체)로 이루어진 리스트입니다. 각 딕셔너리는 특정 구조를 가질 수 있으며, 이 속성 또한 필수입니다.

이 스키마에 따라 JSON 객체를 생성할 때, 반드시 `file_name`과 `data` 속성을 포함해야 하며, `data`는 객체의 리스트로 구성되어야 합니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Bot: 죄송합니다. 처리 중 오류가 발생했습니다.

[시스템] 의도 분석 결과: 'action'

[시스템] 행동 에이전트를 실행합니다...


> Entering new AgentExecutor chain...

Invoking: `load_markdown_and_create_retriever` with `{'file_path': 'brewing_coffee_merged.md'}`


성공: 'brewing_coffee_merged.md' 파일을 읽고 질의응답 준비를 마쳤습니다.
Invoking: `save_json_as